In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.simplefilter('ignore')

In [23]:
df = pd.read_csv(r"E:\Data Science by SRK\Machine_learning\Classification\Gene_data\cleaned_dataset_Gene.csv")

In [24]:
df

,Gene_One,Gene_Two,Cancer_Present
0,4.3,3.9,1
1,2.5,6.3,0
2,5.7,3.9,1
3,6.1,6.2,0
4,7.4,3.4,1
...,...,...,...
2133,5.0,6.5,1
2134,3.4,6.6,0
2135,2.7,6.5,0
2136,3.3,5.6,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gene_One        2138 non-null   float64
 1   Gene_Two        2138 non-null   float64
 2   Cancer_Present  2138 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 50.2 KB


In [26]:
df.isnull().sum()

Gene_One          0
Gene_Two          0
Cancer_Present    0
dtype: int64

# X and y

In [27]:
X = df.drop(columns = ['Cancer_Present'])
y = df['Cancer_Present']

# Train_test_split

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= True)

# Gradient Boost

**Applying Hyperparameter Tuning for identiying best parameters for Gradient Boost**

In [29]:
from sklearn.model_selection import GridSearchCV

# modelling

from sklearn.ensemble import GradientBoostingClassifier
estimator_gb = GradientBoostingClassifier()

# parameters grid
param_grid_gb = {"n_estimators" : [1,5,10,20],
                'learning_rate' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.8, 1]}

# grid search
grid_gb = GridSearchCV(estimator_gb, param_grid_gb, cv = 5, scoring = 'accuracy')
grid_gb.fit(X_train, y_train)

# best parameters for GradientBoostingClassifier model
grid_gb.best_params_

{'learning_rate': 0.8, 'n_estimators': 5}

# Gradient Boost model importance feature

In [30]:
feats_gb = pd.DataFrame(data = grid_gb.best_estimator_.feature_importances_,
                       index = X.columns, 
                       columns = ['Importance'])
imp_features_list_gb = feats_gb[feats_gb['Importance']>0].index.to_list()
imp_features_list_gb

['Gene_One', 'Gene_Two']

# Gradient Boost Classifier

**with imp features and imp parameters**

In [33]:
# important input variables
X_imp = X[imp_features_list_gb]

# train test split
X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(X_imp, y, test_size = 0.2, random_state = True)
# modelling
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators= 5, learning_rate = 0.8)
gb.fit(X_train_gb, y_train_gb)

# Evaluation on train data
ypred_train_gb = gb.predict(X_train_gb)

from sklearn.metrics import accuracy_score
print("Train_accuracy : ", accuracy_score(y_train_gb, ypred_train_gb))

# cross validation
from sklearn.model_selection import cross_val_score
print("cross_val_score : ", cross_val_score(gb, X_train_gb, y_train_gb, cv = 5).mean())

# Evaluation
ypred_test_gb = gb.predict(X_test_gb)
print("Test_accuracy : ", accuracy_score(y_test_gb, ypred_test_gb))


Train_accuracy :  0.9216374269005848
cross_val_score :  0.9128654970760233
Test_accuracy :  0.9228971962616822


In [32]:
%%timeit
gb.fit(X_train_gb, y_train_gb)


81.6 ms ± 7.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
